#Importing necessary libraries

In [ ]:
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bert_extractive_summarizer-0.10.1-py3-none-any.whl (25 kB)


In [ ]:
!pip install transformers
!pip install rouge
from transformers import BertTokenizer
import re
import pandas as pd
import string
from summarizer import Summarizer
from rouge import Rouge
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rouge-1.0.1-py3-none-any.whl (13 kB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Pre-Processing

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Mini Project/dataset.csv')

def remove_date(df):
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    df = df.applymap(lambda x: re.sub(pattern, '', str(x)))
    return df

urls = r'\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def remove_url(df):
  df = df.applymap(lambda x: re.sub(urls, '', str(x)))
  df = df.applymap(lambda x: re.sub('URL: ', '', str(x)))
  return df

foot = 'LIIofIndia: Copyright Policy | Disclaimers | Privacy Policy | Feedback'

def remove_foot(df):
  df = df.applymap(lambda x: re.sub(foot, '', str(x)))
  return df

# X = remove_date(X)
# y = remove_date(y)

# X = remove_url(X)
# y = remove_url(y)

# X = remove_foot(X)
# y = remove_foot(y)

dataset = remove_foot(dataset)
dataset = remove_date(dataset)
dataset = remove_url(dataset)


#Defining model

In [ ]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Generating summaries

In [ ]:
summaries = []
for i in range(0, 5):
    dec_len = len(dataset["Decisions"][i])
    result = model(dataset["Decisions"][i], min_length=20)
    summary = "".join(result)
    summaries.append(summary)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

#Results

In [ ]:
rouge = Rouge()

In [ ]:
rouge1_score = []
rouge2_score = []
rougeL_score = []
for i in range(0, 5):
    scores = rouge.get_scores(summaries[i],dataset["Decisions"][i])
    print("Rouge 1 score of ", i," sample is - ", scores[0]["rouge-1"])
    print("Rouge 2 score of ", i," sample is - ", scores[0]["rouge-2"])
    print("Rouge l score of ", i," sample is - ", scores[0]["rouge-l"])
    rouge1_score.append(scores[0]["rouge-1"]["f"])
    rouge2_score.append(scores[0]["rouge-2"]["f"])
    rougeL_score.append(scores[0]["rouge-l"]["f"])

Rouge 1 score of  0  sample is -  {'r': 0.32673267326732675, 'p': 1.0, 'f': 0.49253730972042775}
Rouge 2 score of  0  sample is -  {'r': 0.2013507809202195, 'p': 0.9636363636363636, 'f': 0.33310055579995407}
Rouge l score of  0  sample is -  {'r': 0.32673267326732675, 'p': 1.0, 'f': 0.49253730972042775}
Rouge 1 score of  1  sample is -  {'r': 0.34075723830734966, 'p': 1.0, 'f': 0.5083056440493483}
Rouge 2 score of  1  sample is -  {'r': 0.21354812061131764, 'p': 0.9699812382739212, 'f': 0.35003384944597804}
Rouge l score of  1  sample is -  {'r': 0.34075723830734966, 'p': 1.0, 'f': 0.5083056440493483}
Rouge 1 score of  2  sample is -  {'r': 0.32539030402629415, 'p': 0.9974811083123426, 'f': 0.4907063159931993}
Rouge 2 score of  2  sample is -  {'r': 0.20635391923990498, 'p': 0.9733893557422969, 'f': 0.34051935037182157}
Rouge l score of  2  sample is -  {'r': 0.32539030402629415, 'p': 0.9974811083123426, 'f': 0.4907063159931993}
Rouge 1 score of  3  sample is -  {'r': 0.305019305019305

In [ ]:
print("Average rouge 1 score is", np.mean(rouge1_score))
print("Average rouge 2 score is", np.mean(rouge2_score))
print("Average rouge l score is", np.mean(rougeL_score))

Average rouge 1 score is 0.4885848705021679
Average rouge 2 score is 0.3296298456027401
Average rouge l score is 0.4885848705021679


In [ ]:
for i in range(0, 5):

    vectorizer = TfidfVectorizer()
    doc_vec = vectorizer.fit_transform([dataset["Headnote"][i]])
    summary_vec = vectorizer.transform([summaries[i]])
    similarity_score = cosine_similarity(doc_vec, summary_vec)

    print("Cosine similarity between sample", i, "document and summary:", similarity_score[0][0])

Cosine similarity between sample 0 document and summary: 0.7190798865444482
Cosine similarity between sample 1 document and summary: 0.8947234310398638
Cosine similarity between sample 2 document and summary: 0.9203805413665784
Cosine similarity between sample 3 document and summary: 0.9464392864908948
Cosine similarity between sample 4 document and summary: 0.857898802755396
